# Importing required Librabries


In [1]:
!pip install git+https://github.com/forrestbao/pyeeg.git
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math
import os
import time

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-rosnpwq4
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-rosnpwq4
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28123 sha256=66839c2dee4fb00136b6199cd9906441b7a663e65400a290c5d1cff460be9344
  Stored in directory: /tmp/pip-ephem-wheel-cache-clru_824/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg


In [2]:
import pandas as pd
import keras.backend as K
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
os.getcwd()
os.chdir('/content/drive/MyDrive/Colab Notebooks/data_preprocessed_python')

In [5]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31]
band = [4,8,12,16,25,45] #5 bands
length_window = 256 # 2 sec
size_step = 16 #Each 0.125 sec 
frequency_sample = 128 #   128 hz
Total_subjects = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']

# Fast Fourier Transform

In [6]:
def Fast_Fourier_Transform (subject, channel, band, length_window, step_size, frequency_sample,c):

    processed = []
    data_frame=open(subject,'rb')

    subject_data = pickle.load(data_frame,encoding = "latin1")  

    for i in range (0,39):
        data = subject_data["data"][i]
        labels = subject_data["labels"][i]
        start = 0;

        while start + length_window < data.shape[1]:
            processed_array = []
            processed_data = [] #meta vector for analysis
            for j in channel:
                X = data[j][start : start + length_window]     #Slice raw data over 2 sec, at interval of 0.125 sec
                Y = pe.bin_power(X, band, frequency_sample)       #FFT over 2 sec of channel j, in sequence of theta, alpha, low beta, high beta, gamma
                processed_data = processed_data + list(Y[0])

            processed_array.append(np.array(processed_data))
            processed_array.append(labels)

            processed.append(np.array(processed_array))    
            start = start + size_step
            
    processed = np.array(processed)
    np.save('/content/drive/MyDrive/Colab Notebooks/DEAP  - saved/s' + str(c), processed, allow_pickle=True, fix_imports=True)

In [ ]:
import glob
FilesTotal = glob.glob("/content/drive/MyDrive/Colab Notebooks/data_preprocessed_python" + "/*.dat")
c=1
for subject in FilesTotal:
    Fast_Fourier_Transform (subject, channel, band, length_window, size_step, frequency_sample,c)
    c = c +1

In [ ]:
X_training = []
Y_training = []
X_testing = []
Y_testing = []

for subjects in Total_subjects:

    with open('/content/drive/MyDrive/Colab Notebooks/DEAP  - saved/s' + subjects + '.npy', 'rb') as file:
      sub_data = np.load(file,allow_pickle=True)
      for i in range (0,sub_data.shape[0]):
        if i % 5 == 0:
          X_testing.append(sub_data[i][0])
          Y_testing.append(sub_data[i][1])
        else:
          X_training.append(sub_data[i][0])
          Y_training.append(sub_data[i][1])

np.save('//content/drive/MyDrive/Colab Notebooks/DEAP  - saved/data_training', np.array(X_training), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/Colab Notebooks/DEAP  - saved/label_training', np.array(Y_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(X_training).shape, np.array(Y_training).shape)

np.save('/content/drive/MyDrive/Colab Notebooks/DEAP  - saved/data_testing', np.array(X_testing), allow_pickle=True, fix_imports=True)
np.save('/content/drive/MyDrive/Colab Notebooks/DEAP  - saved/label_testing', np.array(Y_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(X_testing).shape, np.array(Y_testing).shape)

training dataset: (473539, 70) (473539, 4)
testing dataset: (118405, 70) (118405, 4)
